In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint 
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%Y")

# create an online endpoint 
endpoint = ManagedOnineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MlFlpw model",
    auth_model="key"
)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
from azure.ai.ml.entities import Model , ManagedOnlineDepolyment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model = Model(
    path = "./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model"
)
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)



In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()


In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue":100}
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

In [ ]:
# List endpoint
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)